**Demand Forecasting:**  
- At any given point in time, the optimal price  P∗  depends on a forecast of demand  Dˆ , as well as many other factors,
P∗t=f(Dˆt,…,other factors,…), and is typically the solution to an optimization problem that maximizes a metric such as revenue or profit.  
- A price-optimization algorithm uses a demand forecasting model to predict demand at various price points, and includes business constraints to maximize potential profit.  
- Maximum revenue is an optimization problem, involving dynamic programming.

**Consider the problem of demand estimation for $n$ substituable products.**  
2 important problems:  
- turned-away demand when items are sodl out (**spilled demand**)  
- properly accounting for substitution effects among related items (**recaptured demands**)  

For simplicity, most retail demand forecasts rely on **time-series** models of **observed sales data**, which treat each stock keeping unit (SKU) as receiving an independent stream of requests.  
- if spilled demand is ignored the resulting demand forecasts might be **negatively biased**.  

**Recaptured demand**: substitute products that are available  
- ignoring this leads to an **overestimation bias**. 

**true demand = observed demand + spill demand - recapture demand + other stuff + noise**

Estimating spilled and recaptured demand is called **demand untruncation** or **uncensoring methods**.

One of the most popular is **Expected-Maximization (EM) Algorithm**.  
- works by using alternating steps of computing conditional expected values of the parameter estimates to obtain an expected log-likelihood function (the E-step) and maximizing this function to obtain improved estimates (the M-step).  
- limited to untruncating sales history for individual SKUs and disregard recapture effects from substitute products.

# Walmart Dataset

We will use sales data from Walmart to estimate future demand of departments within stores (think of the stores' aggregate department sales as a proxy for all products' demand in that department, at that store).

A little about deep learning:  
**Dicriminative Deep Learning:** Uber ride time, mostly from consumer perspectives, content filtering, stock prediction, demand forecasting, taking data that's already there. You give the data, the model predicts the label.  
$Y = f(X)$.  

**Generative Deep Learning:** you give a label, and the model generates the data.  
- Ex: giving a description of an image, the model produces the actual image. You give a prompt, ChatGPT writes the essay.  

$X = g(Y)$

Usually:  
Discriminative $\implies$ **Supervised**  
Generative $\implies$ **Unsupervised**

Generative DL is great because it gives businesses consumer-facing products that everyone can appreciate.

In [15]:
import pandas as pd
import os
data_path = 'data/01_part_1/'

**Description of Data - features.csv**  

Contains data related to the store, department, and regional activity for the given dates, such as:  
the store number, the week  
the average temperature in the region  
the cost of fuel in the region  
five anonymous variables (called MarkDown 1, 2, 3, 4, 5) related to promotional markdowns that Walmart is running  
  MarkDown data is not available for all stores at all times  
the consumer price index (CPI)  
the unemployment rate  
a binary variable that indicates whether the week is a special holiday or not  
  the four holidays (Super Bowl, Labor Day, Thanksgiving, Christmas), fall within the following weeks in the data  
  not all holidays are in the data, and dates are in the form: dd-mm-yy  
  Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13  
  Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13  
  Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13  
  Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

In [16]:
features_data = pd.read_csv(data_path + 'features.csv')
features_data.tail()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False
8189,45,2013-07-26,76.06,3.804,212.02,851.73,2.06,10.88,1864.57,NaN,NaN,False


**Pandas**:  stands for panel data (involves entities and time)  
- Creates DataFrames (objects) with rows and columns

**Description of Data - stores.csv**  
Contains anonymized information about the 45 stores  
These variables indicate the type of store and the size of the store

In [18]:
stores_data = pd.read_csv(data_path + 'stores.csv')
stores_data.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [19]:
stores_data['Store'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], dtype=int64)

In [20]:
stores_data['Type'].unique()

array(['A', 'B', 'C'], dtype=object)

**Description of Data - train.csv**  
Contains the labels for the training data  
Covers 2010-02-05 to 2012-11-01  
It includes the store number, the department number, and the date (week)  
Weekly sales for the given department in the given store  
It also includes whether the week is a special holiday or not

In [22]:
train_data = pd.read_csv(data_path + 'train.csv')
train_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [23]:
features_data.shape, stores_data.shape, train_data.shape

((8190, 12), (45, 3), (421570, 5))

In [27]:
data = pd.merge(features_data, stores_data, on=["Store"], how="outer")

data

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,A,151315
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,A,151315
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False,B,118221
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False,B,118221
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False,B,118221
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False,B,118221


In [28]:
data = pd.merge(data, train_data, on=["Store", "Date", "IsHoliday"], how="inner")
data

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size,Dept,Weekly_Sales
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315,1,24924.50
1,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315,2,50605.27
2,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315,3,13740.12
3,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315,4,39954.04
4,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315,5,32229.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,B,118221,93,2487.80
421566,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,B,118221,94,5203.31
421567,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,B,118221,95,56017.47
421568,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,B,118221,97,6817.48


## Splitting Data:  
We will do a **time-series** split into a training and a testing set, where thr training set contains data that occurred **before** the data in the testing set.  
Careful to prevent data leakage, i.e. a lookahead bias, at this step.  

**Why do we split data?**  
We have to simulate the 'real' environment as much as possible before releasing it into the wild. Splitting into training and testing ensures that the model doesn't just work well in research, but also in production/real-world scenarios. We don't want to let the model memorize any testing data, so that we can see if the patterns it memorized in the training set are actually applicable/generalizable on the test set.

In [29]:
data.shape

(421570, 16)

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Date          421570 non-null  object 
 2   Temperature   421570 non-null  float64
 3   Fuel_Price    421570 non-null  float64
 4   MarkDown1     150681 non-null  float64
 5   MarkDown2     111248 non-null  float64
 6   MarkDown3     137091 non-null  float64
 7   MarkDown4     134967 non-null  float64
 8   MarkDown5     151432 non-null  float64
 9   CPI           421570 non-null  float64
 10  Unemployment  421570 non-null  float64
 11  IsHoliday     421570 non-null  bool   
 12  Type          421570 non-null  object 
 13  Size          421570 non-null  int64  
 14  Dept          421570 non-null  int64  
 15  Weekly_Sales  421570 non-null  float64
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 48.6+ MB


In [31]:
data['Date'] = pd.to_datetime(data['Date'])

data.set_index('Date', inplace=True)
data.sort_index(inplace=True)

data

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size,Dept,Weekly_Sales
Date,,,,,,,,,,,,,,,
2010-02-05,1,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315,1,24924.50
2010-02-05,35,27.19,2.784,NaN,NaN,NaN,NaN,NaN,135.352461,9.262,False,B,103681,3,14612.19
2010-02-05,35,27.19,2.784,NaN,NaN,NaN,NaN,NaN,135.352461,9.262,False,B,103681,4,26323.15
2010-02-05,35,27.19,2.784,NaN,NaN,NaN,NaN,NaN,135.352461,9.262,False,B,103681,5,36414.63
2010-02-05,35,27.19,2.784,NaN,NaN,NaN,NaN,NaN,135.352461,9.262,False,B,103681,6,11437.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,13,46.97,3.755,10192.49,364.57,150.0,1714.15,5563.92,131.193097,5.621,False,A,219622,49,7770.71
2012-10-26,13,46.97,3.755,10192.49,364.57,150.0,1714.15,5563.92,131.193097,5.621,False,A,219622,50,1486.00
2012-10-26,13,46.97,3.755,10192.49,364.57,150.0,1714.15,5563.92,131.193097,5.621,False,A,219622,52,4738.93


In [32]:
# An approximate 70/30 Time Series Split

training_data = data.loc[:'2011', :].copy()

testing_data = data.loc['2012':, :].copy()

In [33]:
training_data.tail()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size,Dept,Weekly_Sales
Date,,,,,,,,,,,,,,,
2011-12-30,19,31.65,3.566,6234.22,50438.17,210.34,1457.96,2373.94,136.643258,7.866,True,A,203819,32,12122.50
2011-12-30,19,31.65,3.566,6234.22,50438.17,210.34,1457.96,2373.94,136.643258,7.866,True,A,203819,33,4875.31
2011-12-30,19,31.65,3.566,6234.22,50438.17,210.34,1457.96,2373.94,136.643258,7.866,True,A,203819,34,12897.00
2011-12-30,20,36.35,3.389,6918.65,97740.99,270.38,1648.03,2852.19,212.403576,7.082,True,A,203742,26,9847.01
2011-12-30,20,36.35,3.389,6918.65,97740.99,270.38,1648.03,2852.19,212.403576,7.082,True,A,203742,25,15816.08


In [34]:
testing_data.head()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size,Dept,Weekly_Sales
Date,,,,,,,,,,,,,,,
2012-01-06,26,22.94,3.439,3352.77,17442.27,117.1,468.31,6344.32,136.698129,7.467,False,A,152513,48,117.00
2012-01-06,26,22.94,3.439,3352.77,17442.27,117.1,468.31,6344.32,136.698129,7.467,False,A,152513,49,1920.57
2012-01-06,26,22.94,3.439,3352.77,17442.27,117.1,468.31,6344.32,136.698129,7.467,False,A,152513,52,946.36
2012-01-06,26,22.94,3.439,3352.77,17442.27,117.1,468.31,6344.32,136.698129,7.467,False,A,152513,55,10478.94
2012-01-06,26,22.94,3.439,3352.77,17442.27,117.1,468.31,6344.32,136.698129,7.467,False,A,152513,56,168.69


In [35]:
training_data[['Store', 'Dept']].value_counts().sort_index().unstack()

Dept,1,2,3,4,5,6,7,8,9,10,...,90,91,92,93,94,95,96,97,98,99
Store,,,,,,,,,,,,,,,,,,,,,
1,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,79.0,100.0,100.0,15.0
2,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,17.0
3,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,NaN,33.0,100.0,100.0,97.0,23.0,NaN
4,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,17.0
5,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,NaN,49.0,100.0,100.0,84.0,11.0,NaN
6,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,17.0
7,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,73.0,100.0,100.0,100.0,98.0,NaN
8,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,NaN,100.0,100.0,13.0
9,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,2.0,46.0,100.0,100.0,9.0,11.0,NaN


In [36]:
testing_data[['Store', 'Dept']].value_counts().unstack()

Dept,1,2,3,4,5,6,7,8,9,10,...,90,91,92,93,94,95,96,97,98,99
Store,,,,,,,,,,,,,,,,,,,,,
1,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,22.0
2,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,25.0
3,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,NaN,21.0,43.0,43.0,43.0,13.0,NaN
4,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,25.0
5,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,NaN,25.0,43.0,43.0,43.0,12.0,NaN
6,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,23.0
7,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,43.0,42.0,43.0,43.0,43.0,43.0,1.0
8,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,43.0,43.0,43.0,3.0,43.0,43.0,22.0
9,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,43.0,...,43.0,43.0,43.0,NaN,28.0,43.0,43.0,NaN,3.0,NaN


## Imputing Null Values:  
- We must be very careful to prevent any data leakage, i.e. a lookahead bias, at this step by imputing with values not known at the time the prediction was made.  

- We will fill in missing values in the testing set using the a statistic computed from the training data, which is the only information we would have known at the time of training and predicting on the testing set.

You can also fill with the mean as you go along in the training set, row by row, but we're keeping it simple.

Heuristic: at a *fixed point in time*, what would I know about the data?